In [17]:
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from pathlib import Path
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

In [8]:
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: Daily_Dose_Of_Data_Science_Full_Archive.pdf
  ✓ Loaded 531 pages

Processing: Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow.pdf
  ✓ Loaded 1351 pages

Total documents loaded: 1882


In [9]:
all_pdf_documents

[Document(metadata={'producer': 'macOS Version 14.3 (Build 23D56) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20240712201555Z00'00'", 'title': 'Daily Dose of Data Science Full Archive', 'moddate': "D:20240712201555Z00'00'", 'source': '..\\data\\pdf\\Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'total_pages': 531, 'page': 0, 'page_label': '1', 'source_file': 'Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'file_type': 'pdf'}, page_content='DATASCIENCE\nFREE2024 EDITION\nFULL ARCHIVE\n Avi ChawlaDailyDoseofDS.comDaily Dose ofData Science'),
 Document(metadata={'producer': 'macOS Version 14.3 (Build 23D56) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20240712201555Z00'00'", 'title': 'Daily Dose of Data Science Full Archive', 'moddate': "D:20240712201555Z00'00'", 'source': '..\\data\\pdf\\Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'total_pages': 531, 'page': 1, 'page_label': '2', 'source_file': 'Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'file_type': 'pdf

In [11]:
def text_split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    split_docs = []
    for doc in documents:
        splits = text_splitter.split_text(doc.page_content)
        for i, split in enumerate(splits):
            new_doc = doc.copy()
            new_doc.page_content = split
            new_doc.metadata['chunk_index'] = i
            split_docs.append(new_doc)
    
    print(f"Total chunks created: {len(split_docs)}")
    return split_docs

In [12]:
chunks=text_split_documents(all_pdf_documents)

C:\Users\DEEPAK\AppData\Local\Temp\ipykernel_11460\505711966.py:14: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_doc = doc.copy()


Total chunks created: 3299


In [13]:
chunks[0]

Document(metadata={'producer': 'macOS Version 14.3 (Build 23D56) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20240712201555Z00'00'", 'title': 'Daily Dose of Data Science Full Archive', 'moddate': "D:20240712201555Z00'00'", 'source': '..\\data\\pdf\\Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'total_pages': 531, 'page': 0, 'page_label': '1', 'source_file': 'Daily_Dose_Of_Data_Science_Full_Archive.pdf', 'file_type': 'pdf', 'chunk_index': 0}, page_content='DATASCIENCE\nFREE2024 EDITION\nFULL ARCHIVE\n Avi ChawlaDailyDoseofDS.comDaily Dose ofData Science')

In [19]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [20]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [21]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 3299 texts...


Batches:   0%|          | 0/104 [00:00<?, ?it/s]

Generated embeddings with shape: (3299, 384)
Adding 3299 documents to vector store...
Successfully added 3299 documents to vector store
Total documents in collection: 3299


In [22]:
#Retriever Pipeline From VectorStore
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [23]:
rag_retriever

In [24]:
rag_retriever.retrieve("loss found in neural networks")

Retrieving documents for query: 'loss found in neural networks'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_a9d1daf3_1608',
  'content': 'e\n. \nWhat happens when neural nets are large enough. Set the number\nof neurons to eight, and train the network several times. Notice that\nit is now consistently fast and never gets stuck. This highlights an\nimportant finding in neural network theory: large neural networks\nrarely get stuck in local minima, and even when they do these local\noptima are often almost as good as the global optimum. However,\nthey can still get stuck on long plateaus for a long time.\nf\n. \nThe risk of vanishing gradients in deep networks. Select the spiral\ndataset (the bottom-right dataset under “DATA”), and change the\nnetwork architecture to have four hidden layers with eight neurons\neach. Notice that training takes much longer and often gets stuck on\nplateaus for long periods of time. Also notice that the neurons in\nthe highest layers (on the right) tend to evolve faster than the\nneurons in the lowest layers (on the left). This problem, called the\nv

In [27]:
#RAG Pipeline- VectorDB To LLM Output Generation

import os
from dotenv import load_dotenv
load_dotenv()

#rint(os.getenv("GROQ_API_KEY"))

True

In [30]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [31]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"


In [34]:
rag_retriever.retrieve("End-to-End Machine Learning Project")

Retrieving documents for query: 'End-to-End Machine Learning Project'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_f5a600d8_707',
  'content': 'Chapter 2. \nEnd-to-End Machine\nLearning Project\nIn this chapter you will work through an example project end to end,\npretending to be a recently hired data scientist at a real estate company. This\nexample is fictitious; the goal is to illustrate the main steps of a machine\nlearning project, not to learn anything about the real estate business. Here are\nthe main steps we will walk through:\n1\n. \nLook at the big picture.\n2\n. \nGet the data.\n3\n. \nExplore and visualize the data to gain insights.\n4\n. \nPrepare the data for machine learning algorithms.\n5\n. \nSelect a model and train it.\n6\n. \nFine-tune your model.\n7\n. \nPresent your solution.\n8\n. \nLaunch, monitor, and maintain your system.',
  'metadata': {'creationdate': '2022-12-12T16:42:16+00:00',
   'chunk_index': 0,
   'source': '..\\data\\pdf\\Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow.pdf',
   'total_pages': 1351,
   'file_type': 'pdf',
   'creato

Integration Vectordb Context pipeline With LLM output


In [36]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [37]:
rag_retriever.retrieve("End-to-End Machine Learning Project")

Retrieving documents for query: 'End-to-End Machine Learning Project'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_f5a600d8_707',
  'content': 'Chapter 2. \nEnd-to-End Machine\nLearning Project\nIn this chapter you will work through an example project end to end,\npretending to be a recently hired data scientist at a real estate company. This\nexample is fictitious; the goal is to illustrate the main steps of a machine\nlearning project, not to learn anything about the real estate business. Here are\nthe main steps we will walk through:\n1\n. \nLook at the big picture.\n2\n. \nGet the data.\n3\n. \nExplore and visualize the data to gain insights.\n4\n. \nPrepare the data for machine learning algorithms.\n5\n. \nSelect a model and train it.\n6\n. \nFine-tune your model.\n7\n. \nPresent your solution.\n8\n. \nLaunch, monitor, and maintain your system.',
  'metadata': {'content_length': 686,
   'chunk_index': 0,
   'page': 79,
   'total_pages': 1351,
   'page_label': '80',
   'source_file': 'Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow.pdf',
   'creator': 'calibre 3.48

Enhanced RAG Pipeline Features

In [39]:
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("End-to-End Machine Learning Project", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'End-to-End Machine Learning Project'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Answer: This chapter outlines the steps involved in a typical machine learning project, using a fictional real estate example.  

The steps include:

1. **Understanding the problem**
2. **Gathering data**
3. **Exploring and visualizing data**
4. **Preparing data for algorithms**
5. **Selecting and training a model**
6. **Fine-tuning the model**
7. **Presenting the solution**
8. **Launching, monitoring, and maintaining the system** 


The chapter aims to provide a high-level overview of these steps and introduce fundamental machine learning concepts. 

Sources: [{'source': 'Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow.pdf', 'page': 79, 'score': 0.2998082637786865, 'preview': 'Chapter 2. \nEnd-to-End Machine\nLearning Project\nIn this chapter you will work through an example project end to end,\npretending to be a recently hired data scientist at a real estate company. This\nexample is f

In [41]:
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Look at the big picture of machine learning projects", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'Look at the big picture of machine learning projects'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
will look at the workflow of a typical ML project, discuss the main
challenges you may face, and cover how to evaluate and fine-tune a machine
learning system.
This chapter introduces a lot of fundamental concepts (and jargon) that every
data scientist should know by heart. It will be a high-level overview (it’s the

Part I. 
The Fundamentals of
Machine Learning

or better-quality training data, or perhaps select a more powerful model (e.g.,
a polynomial regression model).
In summary:
You studied the data.
You selected a model.
You trained it on the training data (i.e., the learning algorithm searched
for the model parameter values that minimize a cost function).
Finally, you applied the model to make predictions on new cases (this is
called 
inference
), 
hoping that this model will generalize well.
This is what a typi

Chat with streamlit UI

In [ ]:
import streamlit as st
from typing import List, Dict, Any
import time

# --- Your Advanced RAG Pipeline Class ---
class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(
        self,
        question: str,
        top_k: int = 5,
        min_score: float = 0.2,
        stream: bool = False,
        summarize: bool = False
    ) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]

            # Build prompt
            prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {question}

Answer:"""

            # Stream output simulation
            if stream:
                answer = ""
                for i in range(0, len(prompt), 80):
                    chunk = prompt[i:i+80]
                    answer += chunk
                    yield chunk  # stream chunk to UI
                    time.sleep(0.02)
            else:
                response = self.llm.invoke([prompt])
                answer = response.content

        # Add citations
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Summarize
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Save history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }


# --- Streamlit UI ---
st.set_page_config(page_title="📄PDF Loader Chatbot using RAG", layout="wide")
st.title("PDF Loader Chatbot using RAG")

# Sidebar settings
st.sidebar.header("⚙️ Settings")
top_k = st.sidebar.slider("Top K documents", 1, 10, 5)
min_score = st.sidebar.slider("Minimum similarity score", 0.0, 1.0, 0.2)
stream = st.sidebar.checkbox("Stream Answer", value=False)
summarize = st.sidebar.checkbox("Summarize Answer", value=False)

# Inputs
question = st.text_input("Enter your question:")

if "rag_pipeline" not in st.session_state:
    # TODO: Replace with your retriever & llm
    st.session_state.rag_pipeline = AdvancedRAGPipeline(retriever=None, llm=None)

rag_pipeline = st.session_state.rag_pipeline

if st.button("Ask"):
    if not question.strip():
        st.warning("Please enter a question.")
    else:
        with st.spinner("Retrieving and generating answer..."):
            result = rag_pipeline.query(
                question,
                top_k=top_k,
                min_score=min_score,
                stream=stream,
                summarize=summarize
            )

        st.subheader("💡 Answer")
        st.write(result['answer'])

        if result['summary']:
            st.subheader("📝 Summary")
            st.info(result['summary'])

        if result['sources']:
            st.subheader("📚 Sources")
            for i, src in enumerate(result['sources'], 1):
                st.write(f"**[{i}]** {src['source']} (page {src['page']}, score {src['score']:.2f})")
                st.caption(src['preview'])

        st.subheader("📜 Conversation History")
        for hist in result['history']:
            st.markdown(f"**Q:** {hist['question']}")
            st.markdown(f"**A:** {hist['answer']}")
            if hist['summary']:
                st.markdown(f"**Summary:** {hist['summary']}")
            st.write("---")


2025-09-15 20:52:43.176 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 20:52:43.176 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 20:52:43.549 
  command:

    streamlit run C:\Users\DEEPAK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-15 20:52:43.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 20:52:43.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 20:52:43.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 